In [1]:
from google.colab import files
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
! kaggle datasets download -d vladislavkovalev/day3-lemmed
! unzip day3-lemmed -d data

day3-lemmed.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  day3-lemmed.zip
replace data/lemmed_selfish.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!pip install -q transformers==4.17.0
!pip install -q datasets==2.3.2
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [4]:
data_train = pd.read_csv('/content/data/lemmed_train.csv')
data_selfish = pd.read_csv('/content/data/lemmed_selfish.csv')

In [5]:
id2label = {idx: value for idx, value in enumerate(data_train.columns[4:])}
label2id = {value: idx  for idx, value in enumerate(data_train.columns[4:])}

In [6]:
data_train = pd.concat([data_train, data_selfish])

In [7]:
data_train.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

In [8]:
data_train.shape

(14431, 96)

In [9]:
data_train['labels'] = data_train[data_train.columns[3:]]\
                        .apply(lambda x: x.values.flatten().astype(float), axis=1)

In [10]:
fata_train_cache = data_train.copy()

In [11]:
data_train = data_train[['CELEX_ID', 'Text', 'labels']]

In [12]:
data_train = data_train.rename(
    {"CELEX_ID": "celex_id", "Text": "text", "Directory code": 'labels'},
    axis=1)[['text', 'labels']]
data_train.reset_index(inplace=True)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]

  # encode them
  encoding = tokenizer(text, padding="max_length",
                       truncation=True, max_length=500)

  return encoding

In [14]:
hg_dataset = Dataset(pa.Table.from_pandas(data_train))
encoded_dataset = hg_dataset.map(preprocess_data, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.10)

  0%|          | 0/15 [00:00<?, ?ba/s]

In [15]:
encoded_dataset.set_format("torch")
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12987
    })
    test: Dataset({
        features: ['index', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1444
    })
})

In [16]:
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(data_train['labels'].iloc[0]),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [17]:
batch_size = 16
metric_name = "f1"

In [18]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

In [19]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index. If text, index are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12987
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6496


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.144700,0.055242,0.000000,0.500000,0.000000
2,0.053100,0.048961,0.000000,0.500000,0.000000
3,0.047600,0.039703,0.225336,0.565336,0.131579
4,0.036500,0.033690,0.414719,0.641092,0.284626
5,0.030400,0.030924,0.463889,0.669361,0.337258
6,0.028000,0.028725,0.490926,0.682924,0.365651


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index. If text, index are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1444
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-812
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-812/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-812/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-812/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-812/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index. If text, index are not expected by `BertForSequenceC

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.144700,0.055242,0.000000,0.500000,0.000000
2,0.053100,0.048961,0.000000,0.500000,0.000000
3,0.047600,0.039703,0.225336,0.565336,0.131579
4,0.036500,0.033690,0.414719,0.641092,0.284626
5,0.030400,0.030924,0.463889,0.669361,0.337258
6,0.028000,0.028725,0.490926,0.682924,0.365651
7,0.025400,0.027817,0.525547,0.706821,0.413435
8,0.024800,0.027488,0.525270,0.705501,0.409280


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index. If text, index are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1444
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-5684
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-5684/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-5684/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-5684/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-5684/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, index. If text, index are not expected by `BertForSequ

TrainOutput(global_step=6496, training_loss=0.04405746686047521, metrics={'train_runtime': 10664.6974, 'train_samples_per_second': 9.742, 'train_steps_per_second': 0.609, 'total_flos': 2.6717305957992e+16, 'train_loss': 0.04405746686047521, 'epoch': 8.0})

In [39]:
def predict_labels(text, thres_prob=0.5):
    encoding = tokenizer(text, padding="max_length",
                  truncation=True, max_length=300, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = trainer.model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= thres_prob)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

    return predicted_labels

In [44]:
predicted_labels = predict_labels(text=data_test.reset_index()['Text'][0])

In [55]:
def numbers_to_classes(l=[0,10,20]):
  zero_cl = [0.0] * 93
  for i in l:
    zero_cl[i] = 1.0

  return np.array(zero_cl, dtype=np.float)

In [67]:
dd = []
for i in data_test.reset_index()['Text'].values:
  dd.append(numbers_to_classes([label2id[j] for j in predict_labels(text=i)]))

<ipython-input-55-fbe99fb190a4>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(zero_cl, dtype=np.float)


In [69]:
pd.DataFrame(dd).to_csv('res__.csv', index=False)